In [3]:
from openpyxl import load_workbook
from tqdm import tqdm

with open("new_address_list.txt", "w", encoding='UTF-8') as f:
    
    load_wb = load_workbook("address_data.xlsx", data_only=True)
    # for sheet in load_wb.get_sheet_names():
    for sheet in tqdm(load_wb.sheetnames):
        load_ws = load_wb[sheet]
        col_range = load_ws['A:B']
        if sheet != '세종특별자치시':
            for (sigungoo,doro) in zip(col_range[0][1:],col_range[1][1:]):
                f.write(sheet + " " + sigungoo.value + " " + doro.value+'\n')
        else:
            for doro in col_range[0][1:]:
                f.write(sheet + " " + doro.value+'\n')

100%|██████████| 17/17 [00:01<00:00, 14.48it/s]


In [40]:
import random
import requests
from bs4 import BeautifulSoup

list_path = 'new_address_list.txt'

class AddressGen:
    def __init__(self, list_path):
        print('address generator generated')
        self.base_url = 'https://www.juso.go.kr/support/AddressMainSearch.do?currentPage=1&countPerPage=100&&searchType=HSTRY&searchKeyword='   
        self.list_path = list_path
        self.infile = open(self.list_path, 'r', encoding='utf8')
        self.address_list = [line.strip() for line in self.infile.readlines()]
        self.address_total_len = len(self.address_list)
        self.tmp_list = []

    def __del__(self):
        print('address generator deleted')    
        self.infile.close()
    
    def add_dongho(self, text):
        if text.rfind('(') == -1:
            return text
        dong = str(random.randrange(1,4)) + str(random.randrange(1,10)).zfill(2)
        ho = str(random.randrange(1,13)) + str(random.randrange(1,5)).zfill(2)
        return text[:text.rfind('(')] + f', {dong}동 {ho}호 ' + text[text.rfind('('):]

    def add_ho(self, text):
        ho = str(random.randrange(1,5)) + str(random.randrange(1,5)).zfill(2)
        if text.rfind('(') == -1:
            return text + f', {ho}호'
        else:
            return text[:text.rfind('(')] + f', {ho}호 ' + text[text.rfind('('):]

    def generate(self):
        while len(self.tmp_list) == 0: # 안 뽑힐 수 도 있어서 뽑힐 떄 까지
            idx = i # 0 ~ len - 1
            url = self.base_url + str(self.address_list[idx].strip() .replace(" ","+"))
            response = requests.get(url)
            html = response.text
            soup = BeautifulSoup(html, 'lxml')
            for data in soup.find_all("span",{'class':'roadNameText'}):
                if '도로명정보 보기' in str(data):
                    self.tmp_list.append(data.get_text())
        return_value = self.tmp_list[random.randrange(0,len(self.tmp_list))]
        
        if '아파트' in return_value:
            return_value = self.add_dongho(return_value)
        elif '빌라' in return_value:
            return_value = self.add_ho(return_value)
        elif random.randrange(0,2) == 0:
            return_value = self.add_ho(return_value)

        self.tmp_list = [] # 지워주기
        return return_value

In [41]:
addressgen = AddressGen(list_path)

address generator generated
address generator deleted


In [44]:
a_lst = []

for i in range(len(list_path)):
    al = addressgen.generate()
    a_lst.append(al)

KeyboardInterrupt: 

In [ ]:
import pickle

with open('detail_address_list.pkl', 'wb') as f:
    pickle.dump(a_lst, f)